In [1]:
import os,sys

In [2]:
%pwd

'c:\\Users\\91962\\Desktop\\Machine_learning\\chicken_disease\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\91962\\Desktop\\Machine_learning\\chicken_disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class PrepareCallbacksConfig:
    root_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from Cnn_Classifier.constants import *
from Cnn_Classifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                  config_filepath=CONFIG_FILE_PATH,
                  params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([(model_ckpt_dir)])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [9]:
import urllib.request as request
import tensorflow as tf
import time
from Cnn_Classifier.exception import CustomEx

In [ ]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config=config
    
    # @property
    # def _create_tb_callback(self):
    #     timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
    #     tb_running_log_dir=os.path.join(
    #         self.config.tensorboard_root_log_dir,
    #         f"tb_logs_at_{timestamp}"
    #     )
    #     return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return self._create_ckpt_callbacks
        

In [ ]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callbacks_config()
    prepare_callbacks=PrepareCallback(config=prepare_callbacks_config)
    callback_list=prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise CustomEx(e, sys)